# Ref sobre transfer learning:
https://keras.io/guides/transfer_learning/

https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

https://keras.io/api/applications/#usage-examples-for-image-classification-models


In [1]:
# ! pip install pydot
# ! pip install graphviz


In [2]:
from tensorflow import keras
from tensorflow.keras import models, layers
from tensorflow.keras import optimizers
import os
from tensorflow.keras.utils import image_dataset_from_directory # type: ignore
from tensorflow.keras.metrics import (F1Score, Precision, Recall, CategoricalAccuracy,  # type: ignore
                                      TruePositives, TrueNegatives, FalsePositives, FalseNegatives) 
import yaml
import pickle
import matplotlib.pyplot as plt



import numpy as np
import pandas as pd

#from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras import Sequential, layers, initializers, regularizers, optimizers, metrics 


In [3]:
dicParams = {}
dicParams["model_name"] = "base_03"
dicParams["model_description"]	= "base_03, 150x150, 4Conv, 2MaxPool, 1Dense, 2Dropout, 1Sigmoid, RMSprop(lr=0.001)"

dicParams["batch_size"] = 256
#dicParams["image_size"] = (150,150)
dicParams["image_size"] = (128,128)
dicParams["input_shape"] = dicParams["image_size"] + (3,)
#dicParams["image_size"] = (260,225)
dicParams["epochs"] = 50
dicParams["learning_rate"] = 1e-3

dicParams["steps_per_epoch"] = 32
dicParams["validation_steps"] = 2
dicParams["interpolation"] = "bilinear"
dicParams["color_mode"] = "rgb"

dicParams["labels"] = "inferred"
dicParams["seed"] = 42
dicParams["crop_to_aspect_ratio"] = True

# dicParams["num_classes"] = 6
# dicParams["start_dir"] = "smalldata"
dicParams["num_classes"] = 114
dicParams["start_dir"] = "data"

#IMAGE_SIZE = (244, 244)
#IMAGE_SIZE = (150,150) # - First baseline run
# - First baseline run - BATCH_SIZE = 32


In [4]:
train_path = os.path.join(dicParams["start_dir"], 'train')
val_path = os.path.join(dicParams["start_dir"], 'val')
test_path = os.path.join(dicParams["start_dir"], 'test')


# with the exception of the directory arg all
# other args will be the same for each set
import_kwargs = {
    'labels': dicParams["labels"],
    'label_mode': 'categorical',  # if 'categorical' use 'categorical_crossentropy', if 'int' use 'sparse_categorical_crossentropy'
    'color_mode': 'rgb',
    'batch_size': dicParams["batch_size"],
    'image_size': dicParams["image_size"],
    'seed': dicParams["seed"],
    'interpolation': dicParams["interpolation"],
    'crop_to_aspect_ratio': dicParams["crop_to_aspect_ratio"]
}


# this function returns a tf.data.Dataset object
train_data = image_dataset_from_directory(
    train_path,
    shuffle=True,
    **import_kwargs
)

val_data = image_dataset_from_directory(
    val_path,
    shuffle=False,
    **import_kwargs
)

test_data = image_dataset_from_directory(
    test_path,
    shuffle=False,
    **import_kwargs
)

# making sure all partitions have the same labels in the same order
assert train_data.class_names == val_data.class_names == test_data.class_names


Found 11556 files belonging to 114 classes.
Found 2477 files belonging to 114 classes.
Found 2477 files belonging to 114 classes.


In [5]:
# Generate an object of type tf.data.Dataset 
ds_train = train_data

ds_val = val_data

ds_test = test_data


In [ ]:
dic_base_models = {}

dic_base_models["ResNet50V2"] = keras.applications.ResNet50V2(
    weights="imagenet",
    input_shape= dicParams["input_shape"],
    include_top=False
)
dic_base_models["ResNet50V2"].summary()

dic_base_models["VGG16"] = keras.applications.vgg16.VGG16(
    weights="imagenet",
    #input_shape= (128,128,3),
    input_shape= dicParams["input_shape"],
    include_top=False
)
dic_base_models["VGG16"].summary()

dic_base_models["InceptionResNetV2"] = keras.applications.InceptionResNetV2(
    weights="imagenet",
    input_shape= dicParams["input_shape"],
    include_top=False
)
dic_base_models["InceptionResNetV2"].summary()

dic_base_models["Xception"] = keras.applications.Xception(
    weights='imagenet',  # Load weights pre-trained on ImageNet.
    input_shape= dicParams["input_shape"],
    include_top=False)  # Do not include the ImageNet classifier at the top.
dic_base_models["Xception"].summary()



for base_model in dic_base_models.values():
	base_model.trainable = False

#base_model.trainable = False


In [7]:
augmentation = models.Sequential([layers.RandomFlip(), 
                           layers.RandomRotation(factor=0.3), 
                           layers.RandomZoom(height_factor=0.1, width_factor=0.1),
                           layers.RandomTranslation(height_factor=(-0.1, 0.1), width_factor=(-0.1, 0.1))
                          ])

preprocess = models.Sequential([augmentation,layers.BatchNormalization()])


In [8]:


modmetrics = [metrics.CategoricalAccuracy(name='accuracy'),
                       metrics.AUC(name='AUROC'),
                       metrics.Precision(name='precision'),
                       metrics.Recall(name='recall'),
					   F1Score(average='macro', name="F1Macro"),
]



In [9]:


#depois voltar a mudar o numero de epochs
def compilation(model, metrics, optimizer="RMSprop", learning_rate = dicParams["learning_rate"], epochs = dicParams["epochs"], verbose=1, loss="categorical_crossentropy"):
  if optimizer=="RMSprop":
    optimizer=optimizers.RMSprop(learning_rate=learning_rate)
  elif optimizer=="Adam":
    optimizer=optimizers.Adam(learning_rate=learning_rate)
  elif optimizer=="SGD":
    optimizer=optimizers.SGD(learning_rate=learning_rate)
  elif optimizer=="Nadam":
    optimizer=optimizers.Nadam(learning_rate=learning_rate)
  elif optimizer=="Adadelta":
    optimizer=optimizers.Adadelta(learning_rate=learning_rate)
  elif optimizer=="Adagrad":
    optimizer=optimizers.Adagrad(learning_rate=learning_rate)
  else:
    print("Invalid optimizer")
    return None
  
  model.compile(loss=loss, optimizer=optimizer, metrics=metrics)

  
  history = model.fit(ds_train, validation_data=ds_val, epochs=epochs, verbose=verbose)

  df_hist = pd.DataFrame.from_dict(history.history)
  df_hist["Epoch"] = np.arange(1, len(df_hist) + 1, 1)

  #calculating the f1score for each epoch (train and validation)
  df_hist['f1_calc'] = 2*(df_hist['precision']*df_hist['recall'])/(df_hist['precision'] + df_hist['recall'])
  df_hist['val_f1_calc'] = 2*(df_hist['val_precision']*df_hist['val_recall'])/(df_hist['val_precision']+df_hist['val_recall'])
  
  return df_hist


In [10]:
dic_tranfs_models = {}

for base_model_name in dic_base_models.keys():
	#inputs = keras.Input(shape=(128, 128, 3))
	inputs = keras.Input(shape=dicParams["input_shape"])

	x = preprocess(inputs)
	x = base_model(x, training=False)
	x = keras.layers.BatchNormalization()(x)
	x = keras.layers.Activation("relu")(x)
	x = keras.layers.GlobalMaxPooling2D()(x)

	outputs = keras.layers.Dense(dicParams["num_classes"],activation="softmax", 
									kernel_initializer=keras.initializers.GlorotNormal(seed=dicParams["seed"]))(x)
	modelT = keras.Model(inputs, outputs)

	modelT.summary()
	dic_tranfs_models[base_model_name] = modelT

	# dot_img_file = base_model_name + '.png'
	# keras.utils.plot_model(modelT, to_file=dot_img_file, show_shapes=True)
	


Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_5 (InputLayer)        [(None, 128, 128, 3)]     0         
                                                                 
 sequential_1 (Sequential)   (None, 128, 128, 3)       12        
                                                                 
 xception (Functional)       (None, 4, 4, 2048)        20861480  
                                                                 
 batch_normalization_208 (B  (None, 4, 4, 2048)        8192      
 atchNormalization)                                              
                                                                 
 activation_203 (Activation  (None, 4, 4, 2048)        0         
 )                                                               
                                                                 
 global_max_pooling2d (Glob  (None, 2048)              0     

In [11]:
tfs = {}
epochsfreeze = 5
epochsunfreeze = 5

for modelT_name in dic_tranfs_models.keys():
	modelT = dic_tranfs_models[modelT_name]

	# lstoptimizers = ["SGD", "Adam", "Nadam", "Adadelta", "Adagrad", "RMSprop"]
	lstoptimizers = ["Adagrad", "Adadelta"]
	for opt in lstoptimizers:
		print("Model: ", modelT_name, ", optimizer:", opt, "learning_rate:", dicParams["learning_rate"], "epochs:", epochsfreeze)
		
		res = compilation(modelT, metrics = modmetrics, optimizer=opt, learning_rate = dicParams["learning_rate"], epochs=epochsfreeze)
		tfs[modelT_name + " : " + opt] = res

		# Unfreeze all layers
		modelT.trainable = True
		#modelT.summary()
		# dot_img_file = modelT_name + '_unfreeze.png'
		# keras.utils.plot_model(modelT, to_file=dot_img_file, show_shapes=True)
		res2 = compilation(modelT, metrics = modmetrics, optimizer=opt, learning_rate = dicParams["learning_rate"], epochs=epochsunfreeze)
		tfs[modelT_name + " : " + opt + " : Unfrozen"] = res2

		


Model:  ResNet50V2 , optimizer: Adagrad learning_rate: 0.001 epochs: 5
Epoch 1/5

46/46 [==============================] - 162s 3s/step - loss: 6.3441 - accuracy: 0.0133 - AUROC: 0.5313 - precision: 0.0137 - recall: 0.0011 - F1Macro: 0.0083 - val_loss: 5.4837 - val_accuracy: 0.0194 - val_AUROC: 0.5539 - val_precision: 0.0690 - val_recall: 8.0743e-04 - val_F1Macro: 0.0060
Epoch 2/5
46/46 [==============================] - 160s 3s/step - loss: 5.8838 - accuracy: 0.0214 - AUROC: 0.5716 - precision: 0.0263 - recall: 0.0018 - F1Macro: 0.0114 - val_loss: 5.2845 - val_accuracy: 0.0190 - val_AUROC: 0.5842 - val_precision: 0.0312 - val_recall: 4.0371e-04 - val_F1Macro: 0.0075
Epoch 3/5
46/46 [==============================] - 160s 3s/step - loss: 5.7039 - accuracy: 0.0271 - AUROC: 0.5868 - precision: 0.0496 - recall: 0.0032 - F1Macro: 0.0149 - val_loss: 5.2104 - val_accuracy: 0.0262 - val_AUROC: 0.6044 - val_precision: 0.0488 - val_recall: 8.0743e-04 - val_F1Macro: 0.0111
Epoch 4/5
46/46 [=====

In [12]:
		

for key in tfs.keys():
	print("-- Model : Optimizer: ", key)
	print(tfs[key])



-- Model : Optimizer:  ResNet50V2 : Adagrad
       loss  accuracy     AUROC  precision    recall   F1Macro  val_loss  \
0  6.344058  0.013326  0.531298   0.013684  0.001125  0.008345  5.483694   
1  5.883828  0.021374  0.571639   0.026283  0.001817  0.011369  5.284451   
2  5.703894  0.027085  0.586788   0.049598  0.003202  0.014883  5.210447   
3  5.540602  0.033316  0.603510   0.049133  0.002942  0.018042  5.155463   
4  5.412502  0.036864  0.618502   0.069486  0.003981  0.020078  5.138659   

   val_accuracy  val_AUROC  val_precision  val_recall  val_F1Macro  Epoch  \
0      0.019378   0.553916       0.068966    0.000807     0.005968      1   
1      0.018975   0.584177       0.031250    0.000404     0.007472      2   
2      0.026241   0.604449       0.048780    0.000807     0.011133      3   
3      0.037142   0.618578       0.126761    0.003633     0.017423      4   
4      0.044409   0.630097       0.135802    0.004441     0.018504      5   

    f1_calc  val_f1_calc  
0  0.0020

In [15]:
#save tfs to file
with open("tfs.pkl", "wb") as f:
	pickle.dump(tfs, f)

# save tfs to yaml
with open("tfs.yaml", "w") as f:
	yaml.dump(tfs, f)
	

In [21]:
# Save trained models to disk
for modelT_name in dic_tranfs_models.keys():
	modelT = dic_tranfs_models[modelT_name]
	modelT.save("transf_" + modelT_name + ".h5")


c:\Temp\DL2024Proj\GroupProject\.venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
